In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn import preprocessing
import classes.convert_df as con
from sklearn.model_selection import train_test_split


In [8]:
#データセット作成
# def dataset(self):
print('MYSQLデータを取得します')

gift = con.Main()
# DBからデータ取得
df = gift.convert_df('RESULT_HORSE','')
infoDf = gift.convert_df('RACE','')
# レース結果と競走馬戦績にレース情報を結合
result = pd.merge(left=df, right=infoDf,
            how='left', on='RACE_ID', suffixes=['', '_i'])

# 保存したデータを読み込みレース結果とレース情報を結合
dataDf = result
loadDf = pd.DataFrame()

# 加工対象があったら加工実行
if len(dataDf) > 0:
    print('最新のレース結果に戦績データを付与します')
    # 上記のデータに戦績データを付与

    tqdm_notebook.pandas()
    gift = con.Main()
    # DBからデータ取得
    dfRaceInfo = gift.convert_df('RACE','')
    dfHorseResult = gift.convert_df('RESULT_HORSE','')
    # 競走馬戦績にレース情報を結合
    dfHorseResulti = pd.merge(left=dfHorseResult, right=dfRaceInfo,
                            how='left', on='RACE_ID', suffixes=['', '_i'])
    # 戦績テーブルからゴミを除去
    df = dfHorseResulti.copy()
    columns = []
    for cn in df.columns:
        if '_i' not in cn:
            columns.append(cn)
    dfHorseResulti = df[columns]
    # 引数のデータフレームをコピー
    df = dataDf.copy()
    # 戦績データとする列を_1～_9のサフィックスを付与して一度numpy配列にする
    cols = np.array([dfHorseResulti.add_suffix(f'_{i}').columns for i in range(1, 10)])
    # 列名を1次元配列にreshape
    cols = cols.reshape(-1)

    # 戦績データ生成関数の定義
    def generateHistoricalData(row):
        # 日付とhorseIdの抽出
        dt = row['RACEDATE']
        horseId = row['HORSE_ID']
        # 戦績データを日付とhorseIdで絞り込み
        na = (
            dfHorseResulti[(dfHorseResulti['RACEDATE']<dt)&
                            (dfHorseResulti['HORSE_ID']==horseId)]
                .head(9)        # 直近9レース分に限定
                .to_numpy()     # numpy配列に変換
                .reshape(-1)    # 1次元配列に変換
        )
        # 戦績データのサイズが9レース分無かったら足りないサイズ分nanで埋める
        if na.size < cols.size:
            naNan = np.array([np.nan for i in range(cols.size - na.size)])
            na = np.concatenate([na, naNan])
        return na
    

    # 戦績データを生成してapplyで関数実行
    df[cols] = df.progress_apply(generateHistoricalData, axis=1, result_type='expand')

    dataAddHis = df
    if len(loadDf) >= 0:
        dataAddHis = pd.concat([loadDf, dataAddHis], axis=0)
else:
    dataAddHis = loadDf

dataset = dataAddHis

print('データセット作成処理が終了しました')
    # return dataset

MYSQLデータを取得します
SELECT * FROM RESULT_HORSE;
SELECT * FROM RACE;
最新のレース結果に戦績データを付与します
SELECT * FROM RACE;
SELECT * FROM RESULT_HORSE;


  0%|          | 0/27954 [00:00<?, ?it/s]

C:\Users\2220216\AppData\Local\Temp\ipykernel_2580\1090547431.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[cols] = df.progress_apply(generateHistoricalData, axis=1, result_type='expand')
C:\Users\2220216\AppData\Local\Temp\ipykernel_2580\1090547431.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[cols] = df.progress_apply(generateHistoricalData, axis=1, result_type='expand')
C:\Users\2220216\AppData\Local\Temp\ipykernel_2580\1090547431.py:65: PerformanceWarning: DataFrame is highly fragmented.  This is usually 

データセット作成処理が終了しました


In [59]:
#データセット前処理
# def dataset_Preprocessing(self,dataset):
    #データセットのロード
data = dataset.copy()
#使用する列名を指定
resultCol = [
    'RACEDATE', 'RACE_ID', 'HORSEFRAME', 'HORSENUMBER', 'HORSE_ID', 'GENDER', 'AGE',
        'WEIGHT', 'JOCKEY_ID', 'ODDS', 'POPULAR', 'TORAINER_ID', 'BASE',
        'HORSE_WEIGHT', 'WEIGHT_GAIN_LOSS', 'HANDICAP', 'RANKING','RACENUMBER',
        'GROUND', 'SPIN', 'DISTANCE', 'WEATHER', 'SITUATION', 'PLACE',
        'GRADE', 'LIMIT','HORSE_TOTAL','TIME'
]
recordCol = [
    'RACENUMBER', 'HORSE_TOTAL', 'HORSEFRAME', 'HORSENUMBER', 'ODDS', 'POPULAR', 'RANKING',
    'JOCKEY_ID', 'WEIGHT', 'TIME', 'HORSE_WEIGHT',
    'WEIGHT_GAIN_LOSS', 'GROUND', 'SPIN', 'DISTANCE',
    'WEATHER', 'SITUATION', 'PLACE', 'GRADE', 'LIMIT', 'HANDICAP',
]
#前N走分戦績の列名を生成
recordCol9 = []
for i in range(1, 10):
    tmpList = list(map(lambda x: x + '_' + str(i), recordCol))
    recordCol9 += tmpList
#列名を合体
COLUMNS = resultCol + recordCol9
#データセット
datasetP = dataset[COLUMNS]
datasetP = datasetP[datasetP['RANKING'] != '中止']
datasetP = datasetP[datasetP['RANKING'] != '除外']
datasetP = datasetP[datasetP['RANKING'] != '取消']
datasetP = datasetP[datasetP['RANKING'] != '失格']
datasetP = datasetP[datasetP['RANKING'] != '未定']
datasetP = datasetP[datasetP['RACEDATE'].replace('年','/').replace('月','/').replace('日','')]
    # return datasetP

KeyError: "None of [Index(['2023年07月22日', '2023年07月22日', '2023年07月22日', '2023年07月22日',\n       '2023年07月22日', '2023年07月22日', '2023年07月22日', '2023年07月22日',\n       '2023年07月22日', '2023年07月22日',\n       ...\n       '2024年01月14日', '2024年01月14日', '2024年01月14日', '2024年01月14日',\n       '2024年01月14日', '2024年01月14日', '2024年01月14日', '2024年01月14日',\n       '2024年01月14日', '2024年01月14日'],\n      dtype='object', length=27723)] are in the [columns]"

In [57]:
# エンコード
class Encode:
    # エンコード実行メソッド
    def encoding(self, src, fit=False):
        df = src.copy()
        self.encodeEnvs = self.generateEncodeEnvs()
        for key, env in tqdm(self.encodeEnvs.items()):
            cols = env['cols']
            le = env['encoder']
            if fit:
                # fitの指示があったらデータフレームから対象の値を取り出して実行
                na = df[cols].to_numpy()
                tg = na.reshape(-1).tolist()
                le.fit(tg)
            for col in cols:
                # 欠損データ以外の列を取り出す
                notNull = df[col][df[col].notnull()]
                # エンコード実行してindexをキーにデータフレームに書き込む
                df[col] = pd.Series(le.transform(notNull), index=notNull.index)
                # エンコードした列はcategory列に変換
                df[col] = df[col].astype('category')
        cols = COLUMNS
        for col in cols:
            df[col] = df[col].astype(float)

        return df
    def generateEncodeEnvs(self):
        encodeEnvs = {}
        # horseIdだけは勝手が違うので個別に環境を生成
        dd = {}
        dd['cols'] = ['HORSE_ID']
        dd['encoder'] = preprocessing.LabelEncoder()
        encodeEnvs['HORSE_ID'] = dd
        # それ以外の対象はリスト化してfor文で処理 #カテゴリ変数をラベルエンコード
        cols = ['GENDER', 'TORAINER_ID', 'BASE', 'JOCKEY_ID', 'HANDICAP', 'GROUND',
                'SPIN', 'WEATHER', 'SITUATION', 'PLACE', 'GRADE', 'LIMIT']
        for col in cols:
            dd = {}
            dd['cols'] = [s for s in resultCol if col in s]
            dd['encoder'] = preprocessing.LabelEncoder()
            encodeEnvs[col] = dd
        return encodeEnvs


In [21]:
# #エンコード
# #ラベルエンコーディング関数の定義
# def labelEncode(df, target, recflg=False):
#     #複数列のラベルエンコーディング関数の定義
#     def listEncoder(tdf, le, cols):
#         #データフレームのコピー
#         tdf_ = tdf.copy()
#         #列名から値を取り出す
#         encoList = []
#         for col in cols:
#             encoList += tdf_[col].unique().tolist()
#         #エンコーダーを生成
#         le.fit(encoList)
#         #複数列分ループ
#         for col in tqdm(cols, desc=cols[0]):
#             #欠損データ以外の列を取り出す
#             notNull = tdf_[col][tdf_[col].notnull()]
#             #エンコード実行してindexをキーにデータフレームに書き込む
#             tdf_[col] = pd.Series(le.transform(notNull), index=notNull.index)
#             #エンコードした列はcategory列に変換
#             tdf_[col] = tdf_[col].astype('category')
#         return tdf_, le
#     #データフレームのコピー
#     tdf = df.copy()
#     #ラベルエンコーダーをインスタンス
#     le = preprocessing.LabelEncoder()
#     #戦績かどうかで分岐
#     if not recflg:
#         #リストかどうかで分岐
#         if type(target) == list:
#             #エンコーダーの生成
#             le.fit(tdf[target])
#             #欠損データ以外の列を取り出す
#             notNull = tdf[target][tdf[target].notnull()]
#             #エンコード実行してindexをキーにデータフレームに書き込む
#             tdf[target] = pd.Series(le.transform(notNull), index=notNull.index)
#             #エンコードした列はcategory列に変換
#             tdf[target] = tdf[target].astype('category')
#         else:
#             #戦績以外で複数データだったら複数列エンコードの実行
#             tdf, le = listEncoder(tdf, le, target)
#     else:
#         #戦績データは列名にサフィックスを付与したリストを生成
#         cols9 = [target] + [target + '_' + str(i) for i in range(1, 10)]
#         #複数列エンコードの実行
#         tdf, le = listEncoder(tdf, le, cols9)
#     #データフレームとエンコーダーをreturn
#     return tdf, le

# #データフレームコピー
# df = datasetP.copy()
# #カテゴリ変数をラベルエンコード
# horseList = ['HORSE_ID']
# df, leHorse = labelEncode(df,horseList)
# df, leGender = labelEncode(df,'GENDER')
# df, leTrainer = labelEncode(df,'TRAINER_ID')
# df, leHomeBase = labelEncode(df,'BASE')
# df, lejockey = labelEncode(df,'JOCKEY_ID',recflg=True)
# df, leHandi = labelEncode(df,'HANDECAP',recflg=True)
# df, leType = labelEncode(df,'GROUND',recflg=True)
# df, leDir = labelEncode(df,'SPIN',recflg=True)
# df, leWether = labelEncode(df,'WEATHER',recflg=True)
# df, leCondition = labelEncode(df,'SITUATION',recflg=True)
# df, lePlace = labelEncode(df,'PLACE',recflg=True)
# df, leGrade = labelEncode(df,'GRADE', recflg=True)
# df, leRegulation = labelEncode(df,'LIMIT',recflg=True)
# #量的変数の列名を生成
# numericCols = ['AGE']
# cols1 = ['HORSE_FRAME','HORSE_NUMBER','ODDS','POPULAR','WEIGHT','HORSE_WEIGHT',
#          'WEIGHT_GAIN_LOSS','RANKING','RACENUMBER','DISTANCE']
# cols2 = ['HORSE_TOTAL','RANKING','TIME']
# numericCols += cols1
# cols3 =cols1 + cols2
# for i in range(1,10):
#     numericCols += map(lambda x: x + '_' + str(i),cols3)
# #量的変数に対して片変数を実行
# for col in tqdm(numericCols):
#     df[col] = df[col].astype(float)
    

In [58]:
#学習
enc = Encode()
df = enc.encoding(src=datasetP, fit=True)
#着順から正解列を生成
df['Act'] = df['RANKING'].map(lambda x: 1 if x <= 3 else 0)
#日付をキーに訓練データと検証データに分割
sepdt = '2023年01月01日'
train = df[df['RACEDATE']<sepdt]
test = df[df['RACEDATE']>=sepdt]

#それぞれ教師データと訓練データに分割
train_x = train.drop(['RACEDATE','RANKING','RACE_ID','Act','ODDS','POPULAR'],axis=1)
train_y = train['Act']
test_x = train.drop(['RACEDATE','RANKING','raceId','Act','ODDS','POPULAR'],axis=1)
test_y = train['Act']

import lightgbm as lgb
model = lgb.LGBMClassifier()
model.fit(train_x,train_y)

  0%|          | 0/13 [00:00<?, ?it/s]


KeyError: "None of [Index(['HORSE_ID'], dtype='object')] are in the [index]"

In [ ]:
"""
#学習モデルの保存
import pickle
with open('model.pickle', mode='wb') as f:
    pickle.dump(モデル名,f,protocol=2)

# モデルのオープン
with open('model.pickle', mode='rb') as f:
    clf = pickle.load(f)
    
"""